**Übung Mustererkennung** *WS 2022/23* -- *K. Brandenbusch,  Gernot A. Fink* -- *Technische Universität Dortmund, Lehrstuhl XII, Mustererkennung in eingebetteten Systemen*
___
# Aufgabe 8 (optional): Support Vector Machines - MNIST

Die Leistungsfähigkeit der Support Vector Machines soll nun für das realistischere Szenario der Zeichenerkennung untersucht werden. 
Hierzu soll ein komplettes Klassifikationssystem für den MNIST-Datensatz erstellt werden.

---

Zuerst müssen die benötigten Module importiert werden.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys
import numpy as np
from sklearn import svm

# Uebergeordneten Ordner zum Pfad hinzufuegen, damit das common Package importiert werden kann
if '..' not in sys.path:
    sys.path.append('..')

from common.classification import ClassificationEvaluator
from common.data_provider import DataProvider


#____________________________________
import matplotlib.pyplot as plt  
import matplotlib.cm as cm
    
def show_data(data, width=1):
    """
    Stellt die Bilder in data zeilenweise dar. Nebeneinander werden width-viele
    Bilder angezeigt. Die Gesamtanzahl der Bilder muss so gewaehlt sein, dass in jeder
    Zeile width-viele Bilder dargestellt werden koennen.
    Params:
        data: Darzustellende Bilder als 2D-ndarray. Eine Zeile entspricht einem Bild.
        width: Anzahl der Bilder einer Zeile in der Visualisierung. (default = 1)
    """
    if len(data.shape) == 1:
        data = data.reshape(1, data.shape[0])
        image_count = 1
    else:
        image_count = data.shape[0]

    image = []
    for i in np.arange(width):
        index = np.arange(i, image_count, width)
        column = data[index, :]
        image.append(column.reshape((28 * column.shape[0], 28)))
    image = np.hstack(tuple(image))

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(image, cmap=cm.get_cmap('Greys_r'))
    ax.set_xticks([])
    ax.set_yticks([])
    return ax

---
## Lineare SVM auf Originaldaten

Laden Sie zunächst die MNIST-Daten in Originalrepräsentation (784-D) und trainieren Sie eine lineare SVM mit den Standardparametern von sklearn.

In [5]:
train_data_provider = DataProvider(DataProvider.MNIST_TRAIN)
test_data_provider = DataProvider(DataProvider.MNIST_TEST)
train_data, train_labels = train_data_provider.get_dataset_and_labels()
test_data, test_labels = test_data_provider.get_dataset_and_labels()

new_svm = svm.LinearSVC()
new_svm.fit(train_data,train_labels)
calc_labels_new = new_svm.predict(test_data)

classiEvali = ClassificationEvaluator(calc_labels_new, test_labels)
classiEvali.print_results()

"""
import common.visualization as visualization
import matplotlib.pyplot as plt

_, ax2 = plt.subplots(dpi=150) 
visualization.plot_svm(ax2, train_data, train_labels, new_svm)
"""


Fehlerrate: 16.7; Anzahl falsch-klassifizierte Muster: 167; Anzahl Muster: 1000
Klassenspezifische Fehlerraten
Klasse 0:	Fehlerrate: 7.0;	Anzahl falsch-klassifizierte Muster:   7;	Anzahl Muster: 100
Klasse 1:	Fehlerrate: 1.0;	Anzahl falsch-klassifizierte Muster:   1;	Anzahl Muster: 100
Klasse 2:	Fehlerrate: 21.0;	Anzahl falsch-klassifizierte Muster:  21;	Anzahl Muster: 100
Klasse 3:	Fehlerrate: 26.0;	Anzahl falsch-klassifizierte Muster:  26;	Anzahl Muster: 100
Klasse 4:	Fehlerrate: 11.0;	Anzahl falsch-klassifizierte Muster:  11;	Anzahl Muster: 100
Klasse 5:	Fehlerrate: 14.0;	Anzahl falsch-klassifizierte Muster:  14;	Anzahl Muster: 100
Klasse 6:	Fehlerrate: 16.0;	Anzahl falsch-klassifizierte Muster:  16;	Anzahl Muster: 100
Klasse 7:	Fehlerrate: 16.0;	Anzahl falsch-klassifizierte Muster:  16;	Anzahl Muster: 100
Klasse 8:	Fehlerrate: 30.0;	Anzahl falsch-klassifizierte Muster:  30;	Anzahl Muster: 100
Klasse 9:	Fehlerrate: 25.0;	Anzahl falsch-klassifizierte Muster:  25;	Anzahl Muster: 100


c:\Program Files\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


'\nimport common.visualization as visualization\nimport matplotlib.pyplot as plt\n\n_, ax2 = plt.subplots(dpi=150) \nvisualization.plot_svm(ax2, train_data, train_labels, new_svm)\n'

---
## Lineare SVM auf PCA-reduzierten Daten

Verwenden Sie an Stelle der 784-dimensionalen Originaldaten die Merkmalsrepraesentationen aus Tag 3. Die jeweiligen SVMs sollen auf der Basis einer linearen SVM mit soft-margin erstellt werden. Vergleichen Sie die Ergebnisse.

In [16]:
from common.pca import PCA
pca_object = PCA(train_data)

targetDim = 100 #90% varianz uebrig
(samples_dim_reduced, error) = pca_object.transform_samples(train_data, target_dim=targetDim)
(test_data_dim_reduced, error) = pca_object.transform_samples(test_data, target_dim=targetDim)
samples_dim_reduced = np.real(samples_dim_reduced)
test_data_dim_reduced = np.real(test_data_dim_reduced)

pca_svm = svm.LinearSVC()
pca_svm.fit(samples_dim_reduced,train_labels)
calc_labels_pca = pca_svm.predict(test_data_dim_reduced)

classiEvali = ClassificationEvaluator(calc_labels_pca, test_labels)
classiEvali.print_results()



Fehlerrate: 40.6; Anzahl falsch-klassifizierte Muster: 406; Anzahl Muster: 1000
Klassenspezifische Fehlerraten
Klasse 0:	Fehlerrate: 23.0;	Anzahl falsch-klassifizierte Muster:  23;	Anzahl Muster: 100
Klasse 1:	Fehlerrate: 2.0;	Anzahl falsch-klassifizierte Muster:   2;	Anzahl Muster: 100
Klasse 2:	Fehlerrate: 58.0;	Anzahl falsch-klassifizierte Muster:  58;	Anzahl Muster: 100
Klasse 3:	Fehlerrate: 39.0;	Anzahl falsch-klassifizierte Muster:  39;	Anzahl Muster: 100
Klasse 4:	Fehlerrate: 32.0;	Anzahl falsch-klassifizierte Muster:  32;	Anzahl Muster: 100
Klasse 5:	Fehlerrate: 79.0;	Anzahl falsch-klassifizierte Muster:  79;	Anzahl Muster: 100
Klasse 6:	Fehlerrate: 25.0;	Anzahl falsch-klassifizierte Muster:  25;	Anzahl Muster: 100
Klasse 7:	Fehlerrate: 28.0;	Anzahl falsch-klassifizierte Muster:  28;	Anzahl Muster: 100
Klasse 8:	Fehlerrate: 47.0;	Anzahl falsch-klassifizierte Muster:  47;	Anzahl Muster: 100
Klasse 9:	Fehlerrate: 73.0;	Anzahl falsch-klassifizierte Muster:  73;	Anzahl Muster: 100


c:\Program Files\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


---
## Kernel SVM auf PCA-reduzierten Daten

Variieren Sie die verwendete Kernelfunktion und vergleichen Sie Ihre Ergebnisse.

In [27]:
pca_object = PCA(train_data)

targetDim = 85 #90% varianz uebrig
(samples_dim_reduced, error) = pca_object.transform_samples(train_data, target_dim=targetDim)
(test_data_dim_reduced, error) = pca_object.transform_samples(test_data, target_dim=targetDim)
samples_dim_reduced = np.real(samples_dim_reduced)
test_data_dim_reduced = np.real(test_data_dim_reduced)

pca_svm = svm.SVC(kernel='rbf')
pca_svm.fit(samples_dim_reduced,train_labels)
calc_labels_pca = pca_svm.predict(test_data_dim_reduced)

classiEvali = ClassificationEvaluator(calc_labels_pca, test_labels)
classiEvali.print_results()

Fehlerrate: 18.5; Anzahl falsch-klassifizierte Muster: 185; Anzahl Muster: 1000
Klassenspezifische Fehlerraten
Klasse 0:	Fehlerrate: 15.0;	Anzahl falsch-klassifizierte Muster:  15;	Anzahl Muster: 100
Klasse 1:	Fehlerrate: 1.0;	Anzahl falsch-klassifizierte Muster:   1;	Anzahl Muster: 100
Klasse 2:	Fehlerrate: 21.0;	Anzahl falsch-klassifizierte Muster:  21;	Anzahl Muster: 100
Klasse 3:	Fehlerrate: 29.0;	Anzahl falsch-klassifizierte Muster:  29;	Anzahl Muster: 100
Klasse 4:	Fehlerrate: 24.0;	Anzahl falsch-klassifizierte Muster:  24;	Anzahl Muster: 100
Klasse 5:	Fehlerrate: 22.0;	Anzahl falsch-klassifizierte Muster:  22;	Anzahl Muster: 100
Klasse 6:	Fehlerrate: 11.0;	Anzahl falsch-klassifizierte Muster:  11;	Anzahl Muster: 100
Klasse 7:	Fehlerrate: 17.0;	Anzahl falsch-klassifizierte Muster:  17;	Anzahl Muster: 100
Klasse 8:	Fehlerrate: 19.0;	Anzahl falsch-klassifizierte Muster:  19;	Anzahl Muster: 100
Klasse 9:	Fehlerrate: 26.0;	Anzahl falsch-klassifizierte Muster:  26;	Anzahl Muster: 100


---
## Parametersuche

Evaluieren Sie, wie sich Veränderungen der jeweiligen Kernelparameter auf die entsprechenden Klassifikationsergebnisse auswirken.